In [1]:
import fastai
from fastai.text.all import *
from pathlib import Path

In [2]:
%%time
path = untar_data(URLs.IMDB)

CPU times: user 0 ns, sys: 3.75 ms, total: 3.75 ms
Wall time: 3.19 ms


In [3]:
path

Path('/home/.fastai/data/imdb')

In [4]:
%%time
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

CPU times: user 1.06 s, sys: 75.5 ms, total: 1.14 s
Wall time: 1.14 s


In [5]:
files

(#100000) [Path('/home/.fastai/data/imdb/test/neg/6201_1.txt'),Path('/home/.fastai/data/imdb/test/neg/11881_2.txt'),Path('/home/.fastai/data/imdb/test/neg/11690_1.txt'),Path('/home/.fastai/data/imdb/test/neg/7300_1.txt'),Path('/home/.fastai/data/imdb/test/neg/2931_1.txt'),Path('/home/.fastai/data/imdb/test/neg/11165_4.txt'),Path('/home/.fastai/data/imdb/test/neg/1026_2.txt'),Path('/home/.fastai/data/imdb/test/neg/11499_3.txt'),Path('/home/.fastai/data/imdb/test/neg/3201_1.txt'),Path('/home/.fastai/data/imdb/test/neg/429_3.txt')...]

In [6]:
txt = files[0].open().read(); txt[:75]

"I didn't know what to expect when I started watching this movie, by the end"

In [7]:
txt

"I didn't know what to expect when I started watching this movie, by the end of it I was pulling my hairs out. This was one of the most pathetic movies of this year...in fact, in the last ten years. David Dhawan should just give up his career as a director. I am yet to come across one original script that David Dhawan has worked on. This one was a complete bit y bit rip off Hitch. I have nothing against remakes as such, but this one is just so lousy that it makes you even hate the original one (which was pretty decent). I fail to understand what actors like Salman and Govinda saw in this script. I read somewhere, that this was supposed to be Govinda's comeback vehicle. If thats true, then only God can save his career. Salman just overacted to the hilt. Govinda who I think is an actor of very high caliber was completely wasted. Katrina Kaif and LAra Dutta had nothing to do apart form wearing designer clothes and smiling for no rhyme or reason. Please stay away form this one!"

In [8]:
%%time
spacy = WordTokenizer()
toks = first(spacy([txt]))

print (coll_repr(toks, 30))

(#212) ['I','did',"n't",'know','what','to','expect','when','I','started','watching','this','movie',',','by','the','end','of','it','I','was','pulling','my','hairs','out','.','This','was','one','of'...]
CPU times: user 253 ms, sys: 3.77 ms, total: 256 ms
Wall time: 258 ms


In [9]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [10]:
tkn = Tokenizer(spacy)
print (coll_repr(tkn(txt), 31))

(#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i','started','watching','this','movie',',','by','the','end','of','it','i','was','pulling','my','hairs','out','.','xxmaj','this','was','one'...]


In [11]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [12]:
??replace_rep

Signature: replace_rep(t)
Source:   
def replace_rep(t):
    "Replace repetitions at the character level: cccc -- TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)
File:      /opt/conda/lib/python3.8/site-packages/fastai/text/core.py
Type:      function


In [13]:
coll_repr(tkn('&copy;     Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [14]:
txts = L(o.open().read() for o in files[:2000])

In [15]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz = sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:70])

In [16]:
subword(1000)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


"▁I ▁didn ' t ▁know ▁what ▁to ▁expect ▁when ▁I ▁start ed ▁watching ▁this ▁movie , ▁by ▁the ▁end ▁of ▁it ▁I ▁was ▁p ul ling ▁my ▁ ha ir s ▁out . ▁This ▁was ▁one ▁of ▁the ▁most ▁pa the tic ▁movies ▁of ▁this ▁year ... in ▁fact , ▁in ▁the ▁last ▁t en ▁years . ▁Da v id ▁D ha w an ▁should ▁just ▁give ▁up ▁his ▁care"

In [17]:
subword(200)

"▁I ▁d id n ' t ▁ k n o w ▁w h at ▁to ▁ e x p e c t ▁w h en ▁I ▁st ar t ed ▁w at ch ing ▁this ▁movie , ▁ b y ▁the ▁ en d ▁of ▁it ▁I ▁was ▁p u ll ing ▁ m y ▁h a ir s ▁ o u t . ▁ T h i s ▁was"

In [18]:
subword(1000000)

"▁I ▁didn ' t ▁know ▁what ▁to ▁expect ▁when ▁I ▁started ▁watching ▁this ▁movie , ▁by ▁the ▁end ▁of ▁it ▁I ▁was ▁pull ing ▁my ▁hair s ▁out . ▁This ▁was ▁one ▁of ▁the ▁most ▁pathetic ▁movies ▁of ▁this ▁year ... in ▁fact , ▁in ▁the ▁last ▁ten ▁years . ▁David ▁Dhawan ▁should ▁just ▁give ▁up ▁his ▁career ▁as ▁a ▁director . ▁I ▁am ▁yet ▁to ▁come ▁a cross ▁one"

In [19]:
## numericalization with fastai

In [20]:
toks = tkn(txt)
print (coll_repr(tkn(txt), 31))

(#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i','started','watching','this','movie',',','by','the','end','of','it','i','was','pulling','my','hairs','out','.','xxmaj','this','was','one'...]


In [21]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i'...]

In [22]:
print (toks200[0])

['xxbos', 'i', 'did', "n't", 'know', 'what', 'to', 'expect', 'when', 'i', 'started', 'watching', 'this', 'movie', ',', 'by', 'the', 'end', 'of', 'it', 'i', 'was', 'pulling', 'my', 'hairs', 'out', '.', 'xxmaj', 'this', 'was', 'one', 'of', 'the', 'most', 'pathetic', 'movies', 'of', 'this', 'year', '…', 'in', 'fact', ',', 'in', 'the', 'last', 'ten', 'years', '.', 'xxmaj', 'david', 'xxmaj', 'dhawan', 'should', 'just', 'give', 'up', 'his', 'career', 'as', 'a', 'director', '.', 'i', 'am', 'yet', 'to', 'come', 'across', 'one', 'original', 'script', 'that', 'xxmaj', 'david', 'xxmaj', 'dhawan', 'has', 'worked', 'on', '.', 'xxmaj', 'this', 'one', 'was', 'a', 'complete', 'bit', 'y', 'bit', 'rip', 'off', 'xxmaj', 'hitch', '.', 'i', 'have', 'nothing', 'against', 'remakes', 'as', 'such', ',', 'but', 'this', 'one', 'is', 'just', 'so', 'lousy', 'that', 'it', 'makes', 'you', 'even', 'hate', 'the', 'original', 'one', '(', 'which', 'was', 'pretty', 'decent', ')', '.', 'i', 'fail', 'to', 'understand', 'wh

In [23]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#1880) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','of','and','to','is','it','i','this'...]"

In [24]:
nums = num(toks)[:20]; num;

In [25]:
nums

TensorText([  2,  18,  91,  33, 144,  62,  15, 452,  72,  18, 925, 151,  19,  27,
         11,  68,   9, 135,  13,  17])

In [26]:
' '.join(num.vocab[o] for o in nums)

"xxbos i did n't know what to expect when i started watching this movie , by the end of it"

In [27]:
toks[:20]

(#20) ['xxbos','i','did',"n't",'know','what','to','expect','when','i'...]

In [28]:
nums200 = toks200.map(num)

In [29]:
dl = LMDataLoader(nums200)

In [30]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [31]:
' '.join(num.vocab[o] for o in x[0][:20])

"xxbos i did n't know what to expect when i started watching this movie , by the end of it"

In [32]:
' '.join(num.vocab[o] for o in y[0][:20])

"i did n't know what to expect when i started watching this movie , by the end of it i"

In [33]:
## language model using datablock

In [34]:
get_imdb = partial(get_text_files, folders = ['train', 'test', 'unsup'])

In [35]:
dls_lm = DataBlock(blocks = TextBlock.from_folder(path, is_lm = True),
                   get_items = get_imdb, splitter = RandomSplitter(0.1)).dataloaders(path, path = path, bs = 128, seq_len = 80)

In [36]:
dls_lm.show_batch(max_n = 3)

,text,text_
0,"xxbos xxmaj though i was a kid when "" looker "" came out . xxmaj you got ta absorb yourself into the world of science xxunk - fi ) . xxmaj albert xxunk "" , "" shoot xxmaj the xxmaj moon "" ) , xxmaj susan xxunk xxmaj partridge xxmaj family "" , "" la xxmaj law "" ) , xxmaj james xxmaj coburn , xxmaj dorian xxmaj harewood , and others make of a very fine cast of characters","xxmaj though i was a kid when "" looker "" came out . xxmaj you got ta absorb yourself into the world of science xxunk - fi ) . xxmaj albert xxunk "" , "" shoot xxmaj the xxmaj moon "" ) , xxmaj susan xxunk xxmaj partridge xxmaj family "" , "" la xxmaj law "" ) , xxmaj james xxmaj coburn , xxmaj dorian xxmaj harewood , and others make of a very fine cast of characters in"
1,"as i said , the plot is simple . xxmaj everything else is plain odd . xxmaj the direction is odd , with a weird rotating opening shot to out - of - nowhere sped up sequences . xxmaj the dialouge and the acting is very odd ; odd in a rambling sort of way . xxmaj and the sound track is the oddest thing in the movie , from the weird "" envy "" song that keeps on reappearing","i said , the plot is simple . xxmaj everything else is plain odd . xxmaj the direction is odd , with a weird rotating opening shot to out - of - nowhere sped up sequences . xxmaj the dialouge and the acting is very odd ; odd in a rambling sort of way . xxmaj and the sound track is the oddest thing in the movie , from the weird "" envy "" song that keeps on reappearing to"
2,"indigo kids or feeling spiritually up - lifted … or any of the things the xxmaj producer , xxmaj stephen xxmaj simon , said the film was going to do . xxmaj that 's perhaps their biggest mistake … creating an expectation they did n't deliver on . \n\n xxmaj also , if xxmaj stephen had used the adjective "" extraordinary "" one more time ( i counted at least 6 times ) to describe the film , i would","kids or feeling spiritually up - lifted … or any of the things the xxmaj producer , xxmaj stephen xxmaj simon , said the film was going to do . xxmaj that 's perhaps their biggest mistake … creating an expectation they did n't deliver on . \n\n xxmaj also , if xxmaj stephen had used the adjective "" extraordinary "" one more time ( i counted at least 6 times ) to describe the film , i would have"


In [37]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult = 0.3,
                               metrics = [accuracy, Perplexity()]).to_fp16()

In [38]:
#perplexity = exponential of the loss function

In [40]:
%%time
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.999691,3.904979,0.300210,49.649036,13:55


CPU times: user 13min 46s, sys: 10.9 s, total: 13min 57s
Wall time: 13min 55s


In [41]:
learn.save('1epoch')

Path('/home/.fastai/data/imdb/models/1epoch.pth')

In [43]:
learn.path

Path('/home/.fastai/data/imdb')

In [44]:
model_path = Path.cwd()/"models/"

In [45]:
learn.path = model_path

In [46]:
learn.save("1epoch")

Path('/home/fastai_ch10/models/models/1epoch.pth')

In [47]:
learn.unfreeze()

In [48]:
learn.fit_one_cycle(2, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.713475,3.706157,0.323179,40.697117,14:31
1,3.572263,3.624089,0.333019,37.490570,14:28


In [49]:
learn.save_encoder('finetuned')

In [39]:
## text generation

In [50]:
## loading learner

learn = learn.load("finetuned", strict = False)

In [51]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature = 0.75)
         for _ in range(N_SENTENCES)]

print ("\n".join(preds))

i liked this movie because he " wanted to create a movie that would happen , but it would have to be more one - dimensional . He had to work on it . " In his book Swedish Cinema ,
i liked this movie because it was a bad work , and in it it was not a serious way to make an interesting movie , so it was not a film . As a result , the story of the film is not


In [53]:
dls_clas = DataBlock(blocks = (TextBlock.from_folder(path, vocab = dls_lm.vocab), CategoryBlock),
                     get_y = parent_label,
                     get_items = partial(get_text_files, folders = ['train', 'test']),
                     splitter = GrandparentSplitter(valid_name = 'test')).dataloaders(path, path = path, bs = 128,
                                                                                      seq_len = 72)

In [54]:
dls_clas.show_batch(max_n = 3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg


In [55]:
nums_samp = toks200[:10].map(num)

In [56]:
nums_samp.map(len)

(#10) [231,260,148,335,137,363,155,299,145,60]

In [57]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult = 0.5,
                                metrics = accuracy).to_fp16()

In [60]:
learn.path

Path('/home/.fastai/data/imdb')

In [61]:
model_path = Path.cwd()/"models/"

In [62]:
learn.path = model_path

In [63]:
learn = learn.load_encoder('finetuned')

In [64]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.280186,0.208220,0.916880,00:35


In [65]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4), 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.243239,0.186518,0.927160,00:41


In [66]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4), 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.196988,0.165861,0.936560,00:49


In [67]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4), 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.165762,0.161472,0.939000,01:01
1,0.139669,0.162305,0.940280,01:01
